In [ ]:
import scripts
import tensorflow as tf
import importlib

In [ ]:
importlib.reload(scripts)

### Some Hyperparameters

In [ ]:
NUMBER_OF_CLASSES = 15
IMAGE_SIZE = (64, 64)
IMAGE_SHAPE = IMAGE_SIZE + (3,)

CLASSIFIER_LEARNING_RATE = 0.0001
CLASSFIER_NUMBER_OF_EPOCHS = 10
BATCH_SIZE = 32
OPTIMIZER = tf.keras.optimizers.Adam
LOSS_MEASURE = tf.keras.losses.SparseCategoricalCrossentropy

## Dataloaders

In [ ]:
small_training_dataloader = tf.keras.utils.image_dataset_from_directory("./small_training_dataset/",
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMAGE_SIZE,
                                                                        seed=413)
medium_training_dataloader = tf.keras.utils.image_dataset_from_directory("./medium_training_dataset/",
                                                                         batch_size=BATCH_SIZE,
                                                                         image_size=IMAGE_SIZE,
                                                                         seed=413)
large_training_dataloader = tf.keras.utils.image_dataset_from_directory("./large_training_dataset/",
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMAGE_SIZE,
                                                                        seed=413)
validation_dataloader = tf.keras.utils.image_dataset_from_directory("./validation_dataset/",
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMAGE_SIZE,
                                                                    seed=413)

## Take a Peek

In [ ]:
scripts.peek_into_dataloader(small_training_dataloader)

In [ ]:
small_training_dataloader = small_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
medium_training_dataloader = medium_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
large_training_dataloader = large_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataloader = validation_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

## Training a Classifier

### Base Models

In [ ]:
small_efficientnet_base_model = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=IMAGE_SHAPE) # expects an input to be in the range [0, 255].
medium_efficientnet_base_model = tf.keras.applications.EfficientNetV2M(include_top=False, input_shape=IMAGE_SHAPE)
large_efficientnet_base_model = tf.keras.applications.EfficientNetV2L(include_top=False, input_shape=IMAGE_SHAPE)

### Freeze Pre-Trained Weights

In [ ]:
small_efficientnet_base_model.trainable = False
medium_efficientnet_base_model.trainable = False
large_efficientnet_base_model.trainable = False

### Full Models

In [ ]:
small_efficientnet = scripts.add_classifier(small_efficientnet_base_model, IMAGE_SHAPE, NUMBER_OF_CLASSES)
medium_efficientnet = scripts.add_classifier(medium_efficientnet_base_model, IMAGE_SHAPE, NUMBER_OF_CLASSES)
large_efficientnet = scripts.add_classifier(large_efficientnet_base_model, IMAGE_SHAPE, NUMBER_OF_CLASSES)

In [ ]:
small_efficientnet.compile(optimizer=OPTIMIZER(learning_rate=CLASSIFIER_LEARNING_RATE), loss=LOSS_MEASURE(from_logits=True),
                           metrics=['accuracy'])
medium_efficientnet.compile(optimizer=OPTIMIZER(learning_rate=CLASSIFIER_LEARNING_RATE), loss=LOSS_MEASURE(from_logits=True),
                            metrics=['accuracy'])
large_efficientnet.compile(optimizer=OPTIMIZER(learning_rate=CLASSIFIER_LEARNING_RATE), loss=LOSS_MEASURE(from_logits=True),
                           metrics=['accuracy'])

### Inspection

In [ ]:
print("Number of layers in the small efficientnet model: {}".format(len(small_efficientnet.layers)))
print("Number of layers in the medium efficientnet model: {}".format(len(medium_efficientnet.layers)))
print("Number of layers in the large efficientnet model: {}".format(len(large_efficientnet.layers)))

In [ ]:
small_efficientnet.summary()

In [ ]:
medium_efficientnet.summary()

In [ ]:
large_efficientnet.summary()

### Training the Classifiers

In [ ]:
small_efficientnet_classifier_history = small_efficientnet.fit(small_training_dataloader,
                                                               epochs=CLASSFIER_NUMBER_OF_EPOCHS,
                                                               validation_data=validation_dataloader)

In [ ]:
medium_efficientnet_classifier_history = medium_efficientnet.fit(small_training_dataloader,
                                                                 epochs=CLASSFIER_NUMBER_OF_EPOCHS,
                                                                 validation_data=validation_dataloader)

In [ ]:
large_efficientnet_classifier_history = large_efficientnet.fit(small_training_dataloader, 
                                                    epochs=CLASSFIER_NUMBER_OF_EPOCHS,
                                                    validation_data=validation_dataloader)

### Learning Curves

In [ ]:
scripts.accuracies(small_efficientnet_classifier_history, "Small EfficientNet Training and Validation Accuracy")
scripts.accuracies(medium_efficientnet_classifier_history, "Medium EfficientNet Training and Validation Accuracy")
scripts.accuracies(large_efficientnet_classifier_history, "Large EfficientNet Training and Validation Accuracy")
scripts.losses(small_efficientnet_classifier_history, "Small EfficientNet Training and Validation Loss")
scripts.losses(medium_efficientnet_classifier_history, "Medium EfficientNet Training and Validation Loss")
scripts.losses(large_efficientnet_classifier_history, "Large EfficientNet Training and Validation Loss")

## Fine-Tuning

Only fine tune after the classifer have been trained.

### Some Hyperparameters

In [ ]:
FINE_TUNING_LEARNING_RATE = CLASSIFIER_LEARNING_RATE / 10
FINE_TUNING_NUMBER_OF_EPOCHS = 10
TOTAL_NUMBER_OF_EPOCHS = CLASSFIER_NUMBER_OF_EPOCHS + FINE_TUNING_NUMBER_OF_EPOCHS
NUMBER_OF_LAYERS_TO_FREEZE = []

### Unfreeze All

In [ ]:
small_efficientnet_base_model.trainable = True
medium_efficientnet_base_model.trainable = True
large_efficientnet_base_model.trainable = True

### Freeze a Selected Number of Layers

In [ ]:
for number_of_layer_to_freeze in NUMBER_OF_LAYERS_TO_FREEZE:
    scripts.freeze(small_efficientnet_base_model, number_of_layer_to_freeze)
    scripts.freeze(medium_efficientnet_base_model, number_of_layer_to_freeze)
    scripts.freeze(large_efficientnet_base_model, number_of_layer_to_freeze)

In [ ]:
small_efficientnet.compile(optimizer=OPTIMIZER(learning_rate=FINE_TUNING_LEARNING_RATE), loss=LOSS_MEASURE(from_logits=True),
                           metrics=['accuracy'])
medium_efficientnet.compile(optimizer=OPTIMIZER(learning_rate=FINE_TUNING_LEARNING_RATE), loss=LOSS_MEASURE(from_logits=True),
                            metrics=['accuracy'])
large_efficientnet.compile(optimizer=OPTIMIZER(learning_rate=FINE_TUNING_LEARNING_RATE), loss=LOSS_MEASURE(from_logits=True),
                           metrics=['accuracy'])

### Inspection

In [ ]:
small_efficientnet.summary()

In [ ]:
medium_efficientnet.summary()

In [ ]:
large_efficientnet.summary()

### Training

In [ ]:
small_efficientnet_fine_tune_history = small_efficientnet.fit(train_dataset,
                                                             epochs=TOTAL_NUMBER_OF_EPOCHS,
                                                             initial_epoch=small_efficientnet_classifier_history.epoch[-1],
                                                             validation_data=validation_dataset)

In [ ]:
medium_efficientnet_fine_tune_history = medium_efficientnet.fit(train_dataset,
                                                               epochs=TOTAL_NUMBER_OF_EPOCHS,
                                                               initial_epoch=medium_efficientnet_classifier_history.epoch[-1],
                                                               validation_data=validation_dataset)

In [ ]:
large_efficientnet_fine_tune_history = large_efficientnet.fit(train_dataset,
                                                              epochs=TOTAL_NUMBER_OF_EPOCHS,
                                                              initial_epoch=large_efficientnet_classifier_history.epoch[-1],
                                                              validation_data=validation_dataset)

### Learning Curves

In [ ]:
scripts.accuracies_after_fine_tuning(small_efficientnet_classifier_history,
                                     small_efficientnet_fine_tune_history,
                                     CLASSFIER_NUMBER_OF_EPOCHS,
                                     "Small EfficientNet Training and Validation Accuracy With Fine-Tuning")
scripts.accuracies_after_fine_tuning(medium_efficientnet_classifier_history,
                                     medium_efficientnet_fine_tune_history,
                                     CLASSFIER_NUMBER_OF_EPOCHS,
                                     "Medium EfficientNet Training and Validation Accuracy With Fine-Tuning")
scripts.accuracies_after_fine_tuning(large_efficientnet_classifier_history,
                                     large_efficientnet_fine_tune_history,
                                     CLASSFIER_NUMBER_OF_EPOCHS,
                                     "Large EfficientNet Training and Validation Accuracy With Fine-Tuning")
scripts.losses_after_fine_tuning(small_efficientnet_classifier_history,
                                     small_efficientnet_fine_tune_history,
                                     CLASSFIER_NUMBER_OF_EPOCHS,
                                     "Small EfficientNet Training and Validation Loss With Fine-Tuning")
scripts.losses_after_fine_tuning(medium_efficientnet_classifier_history,
                                     medium_efficientnet_fine_tune_history,
                                     CLASSFIER_NUMBER_OF_EPOCHS,
                                     "Medium EfficientNet Training and Validation Loss With Fine-Tuning")
scripts.losses_after_fine_tuning(large_efficientnet_classifier_history,
                                     large_efficientnet_fine_tune_history,
                                     CLASSFIER_NUMBER_OF_EPOCHS,
                                     "Large EfficientNet Training and Validation Loss With Fine-Tuning")